<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 -- lectures 1 and 2

In the text below there are two kinds of problems:

+ **Problem:**-marked problems, which I intend to solve
  during the lecture. Depending on how fast we progress, I
  may or may not have time to solve them all -- those of the
  problems we have to skip during the lectures are left as
  exercises (see below), but we can discuss them during QA
  sessions.

+ **Exercise:**-marked problems, which I suggest you solve
  yourselves (we can also work on them during the QA
  sessions).
  
-----

# Index

+ [Basic queries](#Basic-queries) (`SELECT-FROM-WHERE`, or SFW)
+ [Set operations](#Set-operations)
+ [Scalar functions and aggregate functions](#Scalar-functions-and-aggregate-functions)
+ [Grouping](#Grouping) (`GROUP BY`)
+ [Subqueries, Views and Common Table Expressions](#Subqueries,-Views-and-Common-Table-Expressions)
+ [Exercises to prepare for lecture 2](Exercises-to-prepare-for-lecture-2)
+ [Redundancy, and the case for splitting up tables](#Redundancy,-and-the-case-for-splitting-up-tables)
+ [Joining tables together](#Joining-tables-together)

-----


This is a [_Jupyter notebook_](https://jupyter.org/), it
contains _cells_ in which we can evaluate program code (I
assume most of you have used notebooks before, but I'll have
a QA-session after the lecture, where you can ask if you
have any questions about it).

Jupyter notebooks have built in support for _Julia_,
_Python_, and _R_ (hence _Ju-Pyt-R_), here's some Python
code:

In [ ]:
def hello(name):
    print(f"hello, {name}!")

def main():
    name = input("What's your name: ")
    hello(name)
    
main()

You can run the code snippet above by clicking somewhere in
the box, and press Shift-Enter.

We're primarily going to run SQL code (see below) in our
notebooks, but I'll also show you some Python code later on
in the course (you don't have to learn Python to take the
course, though).


# Introduction to relational databases

If we were to keep track of all Nobel laureates in a Python
or Java program, and didn't know about relational databases,
we would probably define classes for the laureates, and put
them in lists. We could also define classes for the
categories, and have one list for each category, or have
lists with one element per year, and somehow track all
laureates in that year, or use some kind of
dictionaries/maps. However we chose to keep track of the
data, some searches, insertions and deletions would be easy
to implement, and some would be cumbersome.

In this course, we'll use a technique which at first may
seem too simple to be useful, but which turns out to be
incredibly powerful. We're going to use [_relational
databases_](https://en.wikipedia.org/wiki/Relational_database),
and we'll store the data in 'simple'
[_tables_](https://en.wikipedia.org/wiki/Table_(database)).
Each table looks like a simple spreadsheet -- here is a
table with some Nobel laureates:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border-color:#999;margin:0px auto}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#999;color:#444;background-color:#F7FDFA;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#999;color:#fff;background-color:#26ADE4;}
.tg .tg-e3zv{font-weight:bold}
.tg .tg-9hbo{font-weight:bold;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-e3zv">year</th>
    <th class="tg-9hbo">category</th>
    <th class="tg-9hbo">name</th>
    <th class="tg-9hbo">motivation</th>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Literature</td>
    <td class="tg-yw4l">Tomas Tranströmer</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Physics</td>
    <td class="tg-yw4l">Adam Riess</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Chemistry</td>
    <td class="tg-yw4l">Dan Shechtman</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Physiology or Medicine</td>
    <td class="tg-yw4l">Ralph Steinman</td>
    <td class="tg-yw4l">...</td>
  </tr>
</table>

A _row_ represents an item, and a _column_ represents a
property of the items.

In the example above, each row describes a Nobel laurate,
and for each laureate, we have columns showing what year the
prize was awarded, in what category, the name of the
laureate, and the motivation (not shown here).

The basic idea of relational databases is that all 'cells'
in the table should be simple values (no lists or objects),
and that we can use simple operations from [_relational
algebra_](https://en.wikipedia.org/wiki/Relational_algebra)
to get information from it. We do it using a programming
language which is highly specialized for manipulating and
extracting information, it is called
[SQL](https://en.wikipedia.org/wiki/SQL), which is short
hand for _Structured Query Language_. SQL can be pronounced
as either "S-Q-L", or "sequel".

SQL is divided into several sub-languages:

 + _Data Definition Language_ (_DDL_): constructs used to
   define the tables of a database,

 + _Data Manipulation Language_ (_DML_): statements used to
   query and manipulate data in a database,
   
 + _Transaction Control Language_ (_TCL_): commands used to
   handle transactions (we will return to what a transaction
   is later in the course), and
   
 + _Data Control Language_ (_DCL_): commands used to
   controll access to our data (we'll will not deal with
   them in this course).

This week we'll focus on "Data Manipulation", i.e., ways to
query and modify our databases -- next week we'll look at
how to design and create our databases, and then use DDL to
define our tables.

We'll begin by discussing the following operations from
relational algebra:

 + _selection_: choosing some of the rows of a table

 + _projection_: choosing some of the columns of a table

We will then see various ways to refine and combine queries.


## An actual DBMS

There are many different Relational Database Management
Systems
([RDMBS:es](https://en.wikipedia.org/wiki/Relational_database))
which implements SQL, some of the most prominent are:

 *  [PostgreSQL](https://en.wikipedia.org/wiki/PostgreSQL)
 *  [MariaDB](https://en.wikipedia.org/wiki/MariaDB)
 *  [MySQL](https://en.wikipedia.org/wiki/MySQL)
 *  [Oracle](https://en.wikipedia.org/wiki/Oracle_Database)
 *  [Microsoft SQL Server](https://en.wikipedia.org/wiki/Microsoft_SQL_Server)
 *  [IBM DD2](https://en.wikipedia.org/wiki/IBM_Db2_Family)
 *  [SQLite](https://en.wikipedia.org/wiki/SQLite)

Most of the systems above are
[_client-server_](https://en.wikipedia.org/wiki/Client%E2%80%93server_model)-systems,
i.e., they have one program, a SQL server, which handles the
data, and clients who communicate with the server in various
ways. There are several different kinds of clients:

+ We can run an IDE, which allows us to see our tables in a
  GUI.

+ We can run command line clients (CLI) -- they are text
  based programs who work like typical REPLs, output will
  just be text in a terminal window.

+ We can write scripts which we send to the server, often
  through a CLI.
  
+ We can run a notebook (such as this one), and have it
  communicate with our database.

+ We can write code in a general purpose language, and have
  it communicate with our database.
  
In the course, we'll try all of these methods to access our
databases.

The RDMBS we'll use in the course is
[SQLite](https://en.wikipedia.org/wiki/SQLite), which is a
lightweight but still very powerful system -- it is _by far_
the most used RDBMS, and it's probably already running on
all of your phones and computers (just as an example, if you
use Chrome for browsing, your browsing history is typically
saved in a SQL-database file
`.config/google-chrome/Default/History `, and Mozilla use it
for storing meta-data in Firefox and Thunderbird). It's
actually not a client/server system (instead it is a library
which keeps our databases in files on our computer) -- but
in the course, we'll think of SQLite as if it were a
traditional client/server system, because in many ways, it
behaves as one.

To be able to write SQL queries in this notebook, we first
have to run:

In [ ]:
%load_ext sql

The zip-archive in which this notebook is distributed has a
file `lect01.sqlite` which contains all Nobel Laureates
since 1901 -- to use it in our notebook, we import it with:

In [ ]:
%sql sqlite:///lect01.sqlite

Now we're good to go, we just have to prefix our SQL queries
with `%sql` (one line of SQL) or `%%sql` (several lines of
SQL, this is the form we will use in most cases).


# Basic queries

A simple _SQL query_ can be written as:

~~~{.sql}
SELECT <what we're looking for>
FROM   <what table we're looking in>
~~~


Here `SELECT` is used to select all rows of a given table.

If we're only interesting in some of the rows, and we
normally are, we write:

~~~{.sql}
SELECT <what we're looking for>
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


The latter form is so common that it's got its own acronym:
"SFW" (short for `SELECT`-`FROM`-`WHERE`).

You can see all versions of the `SELECT`-statement in SQLite
on their [documentation for the `SELECT`
statement](https://sqlite.org/lang_select.html) (there are
corresponding pages for other commands).

If we want to see all columns in our rows, we can use

~~~{.text}
SELECT *
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


This is sometimes considered 'sloppy', and we can use a
projection (see above) to get just the columns we're
interested in:

~~~{.text}
SELECT <column 1>, <column 2>, ...
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


Observe that the selection (what rows we're interested in)
is given in the `WHERE` clause, whereas the projection (what
columns we're interested in) is defined in the `SELECT`
clause (the naming is somewhat counter-intuitive).

Our Nobel Database contains the following information for
each laureate:

 *  the _year_ the prize was awarded
 *  the _category_ ('chemistry', 'literature', 'physics',
    'medicine')
 *  the _name_
 *  the _motivation_

Let's use the first form above to see all Nobel prizes which
has been handed out:

In [ ]:
%%sql


This is too much to look through, so let's first limit the
output to 10 rows (once again, look at the [documentation
for `SELECT`](https://sqlite.org/lang_select.html), to see
if you can find out how to do it).

If we limit the number of returned rows, it's often useful
to start listing a number of rows down into the table
(otherwise we could only see the first few rows when we used
`LIMIT`) -- we use `OFFSET` to do that.

We can also select only those prizes awarded in 2013.

In [ ]:
%%sql


Observe that the query returns a new table, we'll soon see
that we can use the returned table in other queries.

**Problem:** _What year did Albert Einstein get his award,
and why?_

In [ ]:
%%sql


The names of the columns in the returned table is shown
above the actual output, if we want to rename any of the
columns in the returned table, we can use an _alias_:

In [ ]:
%%sql


**Problem:** _Who was awarded the physics prize in 1922?_

In [ ]:
%%sql


# Set operations

As was said above, the SQL language is built upon relational
algebra, and sets are a first class citizen of relational
algebra, so we can use set operations such as:

+ _union_ (`UNION`)
+ _intersection_ (`INTERSECT`)
+ _difference_ (`EXCEPT`)

We can use them to combine the results of two or more
queries, _if the queries return tables of the same format_.

**Problem:** _Who were awarded the physics prize in 1922 and
1923?_ Try to solve this problem in at least three different
ways, and see if you can do it using a set operation
(however clumpsy it might be in this case). You can look at
the [documentation](https://sqlite.org/lang_select.html) to
get some inspiration.

In [ ]:
%%sql


In [ ]:
%%sql


In [ ]:
%%sql


There are often several ways of doing things in SQL, and one
of the main points of using SQL is that the database tries
to optimize the operations it needs to fetch our data (there
is some seriously clever code running behind the scenes).

**Problem:** _Who has been awarded the prize in literature since
2010, ordered by name?_

In [ ]:
%%sql


**Problem:** _What year did Winston Churchill win a prize, and in
what category?_

In [ ]:
%%sql


Using `LIKE` in our conditions, we get some rudimentary form
of wildcard matching (some SQL databases allow more advanced
regular expressions, but that's beyond the scope of this
course).

If we want to categorize our output, we can use a `CASE`
statement, it has the general form:

~~~sql
SELECT ..., 
       CASE 
           WHEN ... THEN ...
           WHEN ... THEN ...
           ELSE ...
       END AS <name>
FROM ...
~~~


**Problem:** _Show all laureates in physics with a name beginning
with 'P' -- if they won the prize before 1970 they're
ancient, if the won the prize between 1970 and 2000 they're
veterans, otherwise they're newbies._

In [ ]:
%%sql


## `SELECT` and `SELECT DISTINCT`

**Problem:** _What are the different categories of Nobel prizes?_

In [ ]:
%%sql


Using `SELECT DISTINCT` we only get unique rows in our
output table.


# Scalar functions and aggregate functions

There are some functions we can apply to our values, each
RDBMS supplies their own set of functions -- you can see
some of SQLite's functions
[here](https://sqlite.org/lang_corefunc.html).

**Problem:** _What was the initial letters of the laureates
in year 2000?_ Hint: Use the
[`substr`](https://sqlite.org/lang_corefunc.html#substr)
function (and observe that the first character has index 1).

In [ ]:
%%sql


Here, the number of returned rows is the same as we would
have had if we didn't apply the function.

An _aggregate function_ can be applied to all rows in a
table, _but then returns only one value_.

The standard aggregate functions are:

 + `avg`: calculates the average for a given column
 + `count`: counts the rows in a given table
 + `min`: gets the minimum value of a given column
 + `max`: gets the maximum value of a given column
 + `sum`: calculates the sum of a given column

Observe that these are all functions which operates on
several values, but return a single value. You can see a
list of al SQLite's aggregate functions
[here](https://sqlite.org/lang_aggfunc.html).

**Problem:** _How many laureates were there in year 2000?_

In [ ]:
%%sql


**Problem:** _How many of the laureates has had a first name
beginning with an 'A'?_

In [ ]:
%%sql


**Problem:** _What year was the first Nobel prize awarded?_

In [ ]:
%%sql


**Exercise:** _How many Nobel prizes for chemistry have been
awarded?_

In [ ]:
%%sql


# Grouping

Using `GROUP BY` we can handle rows in groups -- to
understand how it works, lets first look at the following
query:

In [ ]:
%%sql
SELECT    year, category, name
FROM      nobel
WHERE     year = 2013
ORDER BY  category

Here the rows of each category will end up adjacent to each
other, and using `GROUP BY` we insert an invisible divider
between the groups, and perform any aggregate function on
the whole 'group':

In [ ]:
%%sql
SELECT    category, count()
FROM      nobel
WHERE     year = 2013
GROUP BY  category

So, if we apply an aggregate function, such as `count()`, in
a table which we have grouped, _it will be applied to each
group_, not to the whole table. Instead of getting one
`count()` for the whole table (it would be a single value),
we get one `count()` for each group (as above).

If we add `name` in the first line, we get a somewhat
arbitrary result:

In [ ]:
%%sql
SELECT    category, count(), name
FROM      nobel
WHERE     year = 2013
GROUP BY  category

The category and count is correct, but only one name is
shown for each category.

The 'problem' is that we only get one row per group in the
output, and that there may be several laureates in each
group -- our query will return one of them in a seemingly
haphazard manner. We can concatenate all names in the group
using the
[`group_concat`](https://sqlite.org/lang_aggfunc.html#groupconcat)-function:

In [ ]:
%%sql
SELECT    category, count(), group_concat(name)
FROM      nobel
WHERE     year = 2013
GROUP BY  category

There is no problem displaying `category` in the
`SELECT`-statement above, we get a value which we know is
the same for each row in the group (by definition, since
that's what we grouped by).


If we're only interested in those categories with less than
three laureates, we use `HAVING` to select only _groups_
with a given property:

In [ ]:
%%sql
SELECT    category, count(), group_concat(name)
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3

This corresponds to a `WHERE` statement, but it applies to
groups, not to individual rows (as `WHERE` does) -- so,
_`WHERE` and `HAVING` have similar effects (they somehow
narrow a search), but they're absolutely not
interchangable!_

**Important** (and often misunderstood): In the query above
we first have a `WHERE` statement to select some rows from
the whole table, and then group the resulting selection.
_Every time we have both a `WHERE` and a `HAVING` in the
same query, we must first use `WHERE` to select rows we can
group, and then use `HAVING` to select groups._ We can use
`WITH` statements or subqueries (see below) if we want to
have it the other way around.


**Problem:** _How many laureates are there in each
category?_

In [ ]:
%%sql


**Problem:** _Show the categories with more than 200
laureates?_

In [ ]:
%%sql


**Exercise:** _How many laureates were there each year
between 1920 and 1930?_

In [ ]:
%%sql


**Exercise:** _Which years have seen more than 9 laureates?_

In [ ]:
%%sql


**Exercise:** _Which have been the 20 years with most
laureates?_ (We don't need to be precise in case of ties.)

In [ ]:
%%sql


## Olympic Games

To spice things up a bit, I've also included a table with
all olympic games since 1896 -- the table `olympics`
contains the columns:

+ `year`
+ `city`
+ `country`
+ `continent`
+ `season`
+ `ordinal_number`

If we look carefully at this table, we can find some
unnecessary repetition, we will soon address this problem
(but for now, we'll let it pass).

**Problem:** _How many olympic games have each continent
hosted?_

In [ ]:
%%sql


**Problem:** _When was the first olympic games in each continent?_

In [ ]:
%%sql


**Problem:** _Which countries have hosted the summer
olympics more than once?_

In [ ]:
%%sql


**Exercise:** _List the continents in descending order by
the number of times they've hosted the summer olympics_

In [ ]:
%%sql


**Problem:** _Show a 'histogram' (no actual diagram, just
the counts) over the the initial letter of the names of all
Nobel laureates_

In [ ]:
%%sql


We can group by more than one column, by inserting invisible
borders between all combinations of the given column values:

**Problem:** _Show a 'histogram' over the the initial letter
of the names of all Nobel laureates, **for each category**_

In [ ]:
%%sql


**Problem:** _Has anyone won more than one Nobel prize?_ We
can assume the names of the laureates are unique (so far
they are!).

In [ ]:
%%sql


**Problem:** _Has anyone won more than one Nobel prize in
the same category?_

In [ ]:
%%sql


**Problem:** _Has anyone won Nobel prizes in different
categories?_

In [ ]:
%%sql


There is another SQL mechanism, [_window
functions_](https://sqlite.org/windowfunctions.html), which
allows us to partition our rows into groups -- we will
discuss window functions later in the course.



# Subqueries, Views and Common Table Expressions

As we noted above, the result of a `SELECT`-statement is
itself a table, and we can use such a table inside other
statements.

One very useful pattern is:

~~~sql
SELECT ...
FROM   ...
WHERE  ... IN
       (SELECT ...
        FROM ...
        WHERE ...)
~~~


The second query is called a _subquery_.

We'll use a subquery to find all literature laureates who
split their prizes, we begin with a regular query:

**Problem:** _Which years were the Nobel prize for
literature split?_

In [ ]:
%%sql
SELECT    year
FROM      nobel
WHERE     category = 'literature'
GROUP BY  year
HAVING    count() > 1

... and now we use the result of that query to find out what
we're really looking for:

**Problem:** _Which literature laureates split their prizes?_

In [ ]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN (
              SELECT    year
              FROM      nobel
              WHERE     category = 'literature'
              GROUP BY  year
              HAVING    count() > 1)

This can be simplified by using either of two ways to define
'temporary tables':

+ _Views_, and
+ _Common Table Expressions_ (a.k.a. a CTEs, or `WITH`
  statements).

They are the result of a `SELECT` statement, so for our
problem above we could have created a view with the `CREATE
VIEW` statement:

In [ ]:
%%sql
CREATE VIEW shared_literature_prize(year) AS
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1

We can then use this view as if it was a regular table, in:

In [ ]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN (
              SELECT    year
              FROM      shared_literature_prize)

Since `shared_literature_prize` has only one column, we can
make the query even simpler (but beware that not all
databases accept this simplification, and it only works in
some cases):

In [ ]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN shared_literature_prize

A view will stay around until we explicitly drop it:

In [ ]:
%%sql
DROP VIEW shared_literature_prize

Common table expressions are in many ways similar to views,
but they are sometimes more convenient:

+ they're defined as part of a `SELECT` statement (so there
  is nothing to drop afterwards),
+ since they're part of a `SELECT` statement, we only need
  one statement (which will become useful when we call our
  database remotely, we'll return to that later in the
  course), and
+ they can be defined recursively (we'll return to that
  later in the course).

Using a CTE, the solution to the problem above becomes:

In [ ]:
%%sql
WITH shared_literature_prize(year) AS (
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1
)
WHERE     category = 'literature'
          AND year IN (
              SELECT    year
              FROM      shared_literature_prize)

or just (since our CTE has only one column):

In [ ]:
%%sql
WITH shared_literature_prize(year) AS (
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1
)
WHERE     category = 'literature'
          AND year IN shared_literature_prize

**Exercise:** _Show the years and categories for recurring
laureates (i.e., laureates who has won more than once) --
use a CTE to do it._

In [ ]:
%%sql


**Exercise:** _Who has won the literature prize in a year
when at least one chemistry laureate had a name beginning
with 'L'?_ First try to solve this with a regular subquery,
and then rewrite it using a CTE.

In [ ]:
%%sql


We saw above that we can't have another `WHERE` after the
`HAVING` clause:

In [ ]:
%%sql
SELECT    category, count() AS cnt
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3
WHERE     cnt > 1        -- not allowed!

but we can make our 'grouping query' into a subquery, and
have another `WHERE` in the outer query

In [ ]:
%%sql
SELECT category, cnt
FROM (
    SELECT    category, count() AS cnt
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3)
WHERE cnt > 1

A somewhat tidier way of expressing this is to use a
`WITH`-statement:

In [ ]:
%%sql
WITH category_count(category, cnt) AS (
    SELECT    category, count()
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3
)
SELECT category, cnt
FROM   category_count
WHERE  cnt > 1

### Correlated subqueries

Another form of subquery is:

~~~sql
SELECT ...,
       (SELECT ...
        FROM ...
        WHERE ...)
FROM   ...
~~~


This works if the subquery produces one result, such as when
we use an aggregate function. As an example, solve the
following problem:

**Problem:** _List the names of all laureates who has the
longest name of all laureates in their category (in case of
ties, all should be listed) -- order by category._

Here we can use a subquery which is 'run' for each row in
our main query:

In [ ]:
%%sql
SELECT  category, year, name
FROM    nobel AS outer_nobel
WHERE   length(name) = (
            SELECT max(length(name))
            FROM   nobel
            WHERE  category = outer_nobel.category)
ORDER BY category

This is called a _correlated subquery_ (since we refer to
the enclosing query inside it). We use an alias to
distinguish between the nobel table in the outer query and
the nobel table in the subquery (it's the same table, but we
'iterate' through it separately).

BTW, we could have skipped the `AS` in

~~~sql
...
FROM    nobel AS outer_nobel
...
~~~


and just written:

~~~sql
...
FROM    nobel outer_nobel
...
~~~


(personally I think it's a bit less legible, but YMMV, and
it's quite common).



## Exercises to prepare for lecture 2

Some of these exercises have answers right below, try to
solve them yourself before looking at the answer.

**Exercise:** _Write a query to find out who has shared the
chemistry prize with exactly one other laureate in years
when the summer olympics were held in Europe?_

In [ ]:
%%sql


**Answer:** I'll solve this exercise during lecture 2.


**Exercise:** _Above we said that there is some unnecessary
repetition in the olympics database, in what way?_

**Answer:** It is pretty obvious that countries never change
continents (or at least it would literally take _eons_ of
time to do so...), so repeating the fact that France is in
Europe several times ...

In [ ]:
%%sql
SELECT    country, continent, season, year, city
FROM      olympics
WHERE     country = 'France'
ORDER BY  year

... is _redundant_.

And this redundance not only creates unnecessary repetition
(and thus wastes disk space), it also introduces a risk of
failures when we want to modify our table (we'll return to
that later).

The 'problem' here is that we always get the same answer
when we ask which continent a given country is in -- this is
what we will call a _functional dependency_ when we get back
to it in a few weeks time (when we talk about
_normalization_).

The way we're going to solve the problem is to remove the
continents from the `olympics`-table, and use another table,
a kind of 'look-up'-table, to find the continent for each
country -- we're going to do exactly that during this
lecture.

There is another potential redundance in our `olympics`
table -- every time the game has taken place in, say, Paris,
it has been in France:

In [ ]:
%%sql
SELECT    city, group_concat(year || ": " || country, ", ")
FROM      olympics
GROUP BY  city
HAVING    count() > 1
ORDER BY  city

So, it does look as if we could extract the countries as
well, and put them into a seperate 'look-up'-table, but if
Sarajevo were to hold the olympics in the future, it
wouldn't be in Yugoslavia anymore, and Moscow is no longer
in the Soviet Union, so just extracting the countries as we
did with continents above would not work. There is a kind of
database called _temporal databases_, which is designed to
handle cases such as this, but we'll only briefly touch upon
them in this course.

And there is another reason for keeping both name and
country: the names of cities aren't globally unique, there
is a Paris in Texas, and a Moscow in Kansas, so the country
helps us identify the cities uniquely.


**Exercise:** _How could we add information about birth
dates, and birth cities to our Nobel laureates?_

**Answer:** Since each laureate has one birth date, and one
birth city, we can just add two more columns, `birth_date`
and `birth_city` to our `nobel` table.


**Exercise:** _How could we add information about academic
affiliations for the Nobel laureates?_ Some of the laureates
have many affiliations, and some have none -- and observe
that the 'cells' of our table must only contain simple
values, so no lists or objects. Don't spend to much time to
come up with a solution for this -- we haven't yet discussed
the mechanism we're going to use to solve it -- but it's a
good thing if you can see the limitations of what we've seen
so far.


**Answer:** SQL allows us to use the value `NULL` when a
value is missing, so for laureates without any affiliation,
we can just use `NULL` (although doing it is somewhat
controversial, and we can define our tables in a way which
prohibits it -- more about that later).

But handling laureates with several affiliations is more
difficult. One very primitive way of doing it, would be to
add several columns: `affiliation_1`, `affiliation_2`, ...,
but it's not only ugly, it only works if we limit the number
of affiliations to a fixed number.

It turns out we can solve this problem in a manner similar
to how we handled continents above, we can add another table
(a 'look-up'-table if you like), but this time the look-up
table would have all affiliations for each laureate, so it
could have several rows for each laureate.


# Redundancy, and the case for splitting up tables

The term
[redundancy](https://en.wikipedia.org/wiki/Redundancy_(linguistics))
can be defined in many contexts, in linguistics (and in
general) it refers to information which is expressed more
than once. It is sometimes desirable (the human body has an
enormous amount of redundancy, and that's what enable us to
self-heal), but often it is a source of confusion and
errors.

So, in this course, we'll generally try to avoid
redundancies when we design our databases, and we're going
to see two completely different way of doing it:

+ proper _ER-modeling_ (next week), and
+ _normalization_ (week 4).

Our table of Olympic Games looks like this:

In [ ]:
%%sql
SELECT *
FROM   olympics

We can use SQL to create a better version of our olympic
database with the following lines:

In [ ]:
%%sql
DROP TABLE IF EXISTS continents;
CREATE TABLE continents (
  country   TEXT,
  continent TEXT NOT NULL,
  PRIMARY KEY (country)
);

INSERT OR IGNORE
INTO   continents(country, continent)
SELECT country, continent
FROM   olympics;

DROP TABLE IF EXISTS better_olympics;
CREATE TABLE  better_olympics (
  year            INT,
  city            TEXT,
  country         TEXT,
  season          TEXT,
  ordinal_number  TEXT,
  PRIMARY KEY (year, season)
);

INSERT
INTO   better_olympics(year, city, country, season, ordinal_number)
SELECT year, city, country, season, ordinal_number
FROM   olympics;

ALTER TABLE olympics
RENAME TO   redundant_olympics;
ALTER TABLE better_olympics
RENAME TO   olympics;

Now we don't have to enter the continent the next time
Athens, Paris or London is awarded the games.

Our two tables look like this:

In [ ]:
%%sql
SELECT *
FROM   olympics;

In [ ]:
%%sql
SELECT *
FROM   continents;

We've avoided some redundancy, but now we need a way to
combine the information in our two tables, and that's what
_joins_ are for.


# Joining tables together

To introduce joining, we're going to use a classic example:
a database for handling college applications. In it a number
of students applies for various majors at different colleges
-- we want to keep track of:

 + student id (similar to stil-id)
 + student name
 + student's grade average (gpa)
 + the size of the student's highschool
 + the names of the colleges
 + the state for each college
 + the enrollment for each college
 + the major applied for
 + the decision ('Y' for accepted, 'N' otherwise)

One way of doing this would be to use one big table:

In [ ]:
%%sql
SELECT *
FROM   big_college

But, just as with our olympics database above, this would
give us a lot of redundance, so we'd like to split this big
table into several related tables.

**Exercise:** In lectures 7 and 8 we'll discuss the theory
and practice of splitting up tables -- for now, suggest
tables which you think would make working with our data
easier (don't look further down on this page until you've at
least tried).

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

**Solution:** We can have one table for the students:

In [ ]:
%%sql
SELECT *
FROM   students
LIMIT 4

One for the colleges:

In [ ]:
%%sql
SELECT *
FROM   colleges
LIMIT 4

And one for all collage applications made by the students:

In [ ]:
%%sql
SELECT *
FROM   applications
LIMIT 4

The `applications` table uses `s_id` instead of `s_name`,
since we could have several students with the same name (we
actually have two 'Amy') -- `s_id` is guaranteed to be
unique (we'll talk _much_ more about uniqueness next week).

Now assume we want to display all applications, _with the
names of the students_ (not just the student id). We then
need a way to combine information in the `applications`
table with information in the `students` table, and we'll do
it with a
[_join_](https://en.wikipedia.org/wiki/Join_(SQL)).

The SQL standard specifies five different kinds of joins:

 + _cross join_
 + _inner join_
 + _left outer join_
 + _right outer join_
 + _full outer join_

## Cross join

The most primitive of the joins is the cross join -- we can
write it as:

~~~
SELECT      *
FROM        a 
CROSS JOIN  b
~~~


Doing this, we get all combinations of rows from each of
table `a` and `b` (a.k.a. the [_Cartesian
product_](https://en.wikipedia.org/wiki/Cartesian_product))
-- the rows will contain all attributes from both tables
(unless we make a projection).

If we do it with our `applications` and `students` tables we
get:

In [ ]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students

The number of rows in this table is the product of the
number of rows in each table, and most of the rows in the
combined table are totally uninteresting, since the `s_id`'s
doesn't have anything to do with each other (e.g., in one
line the fact that a student with `s_id` 123 has applied to
CS at Stanford is combined with the fact that there is a
student Bob with `s_id` 234 who has a `gpa` of 3.6).

The only rows of interest to us are those where the student
id from the `applications` table is the same as the student
id from the `students` table, and we can express that as:

In [ ]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
WHERE       applications.s_id = students.s_id

or, using aliases, so we don't have to write long table
names:

In [ ]:
%%sql
SELECT      *
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

We can clean up the output using a projection:

In [ ]:
%%sql
SELECT      s.s_name, a.c_name, a.major
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

or even

In [ ]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

(we only need to prefix column names when there would
otherwise be an ambiguity).

There is a shortcut for cross joins, we can just put a comma
between the two tables we want to join:

In [ ]:
%%sql
SELECT s_name AS name, c_name AS college, major
FROM   applications AS a, students AS s
WHERE  a.s_id = s.s_id

This works, but there are much more elegant ways to write
this query.


## Inner joins

An _inner join_ combines two tables into a new table (it's
not actually saved as a table) -- it does this by creating
combined rows only when rows from the two tables 'match'
each other. In this case it's the `s_id` columns which
should match, so we write:

In [ ]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a 
INNER JOIN  students AS s 
ON          a.s_id = s.s_id

Logically it can be seen as the equivalent of a cross join
and then a selection, but the database engine normally use
algorithms which are much faster and requires much less
memory when we define an inner join. Once you get used to
them, inner joins will feel much better to use than cross
joins (although, in theory, the database should try to
optimize all queries, and _might_ find an efficient way to
execute even cross joins).

Inner joins are the default joins, so we can omit the word
`INNER`, and just write `JOIN`:

In [ ]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications AS a
JOIN    students AS s
ON      a.s_id = s.s_id

The condition on which we join tables is often called the
_join predicate_, and a join where the join predicate is an
equality test, such as here, is sometimes called an
_equi-join_. There is a special form of join when the
columns we're comparing in an equi-join have the same name:

In [ ]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
USING   (s_id)

One benefit from using `USING` is that the columns we join
over will not be duplicated, we get only one of them in the
output.

I'd say this is the preferred way of writing this query.

There is an even simpler way to write it, using what's
called a _natural join_, but it's error-prone, and you're
recommended not to use it. It's written as

In [ ]:
%%sql
SELECT        s_name AS name, c_name AS college, major
FROM          applications
NATURAL JOIN  students

and it joins the tables using an equi-join for all columns
with coinciding names -- _this could create big problems if
there are attributes in the tables which just happen to
coincide without us realizing it!_ If we use the `JOIN` ...
`USING` instead, we explicitly declare on which attributes
to join, so we guard ourselves against accidental column
name collisions.

Observe that we can use selections just as before when we
join tables. 

**Problem:** _Write a SQL query which shows all applications
as above, but only for students applying for CS at
Stanford._

In [ ]:
%%sql


**Problem:** _Write a SQL query which shows the average gpa
for students who have applied for each college._

This time it's OK to count applicants with several
applications several times (so it becomes some kind of
weighed average).

In [ ]:
%%sql


Using a CTE (`WITH` statement), we can solve this problem in
a more satisfactory way (not counting grades more than
once):

In [ ]:
%%sql


We can join a table with itself, write a query which finds
all pairs with the same Grade Point Average (GPA):

We can do it either with a cross join:

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students AS s1, students AS s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

or with an inner join:

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s2.s_id, s2.s_name, gpa
FROM   students AS s1 
JOIN   students AS s2 
USING  (gpa)
WHERE  s1.s_id < s2.s_id

Joining a table with itself is called a _self join_.
       
**Exercise:** Show the student names and majors for all
applications to Stanford:

In [ ]:
%%sql


**Exercise:** Show the average `size_hs` for applications to
the different colleges, order by descending size:

In [ ]:
%%sql


We can apply joins in several steps, each time combining
what we've previously joined with another table (the result
of which is a new table, which can be joined over and over
again...).

**Exercise:** _Show the names of all students who have
applied for a college in California -- also show the college
and major._

In [ ]:
%%sql


In lab 1 you'll get plenty of exercise in joining.


### Using CTEs and inner joins to simplify queries

If we often use some query as part of other queries, we can
save it using a _view_:

In [ ]:
%%sql
DROP VIEW IF EXISTS application_info;
CREATE VIEW application_info(s_id, name, major, college, state) AS
  SELECT s_id, s_name, major, c_name, state
  FROM   applications
  JOIN   students
  USING  (s_id)
  JOIN   colleges
  USING  (c_name);

SELECT DISTINCT name, state
FROM   application_info
WHERE  s_id IN (123, 234, 456);

In lab 1 there are a couple of queries which would be easier
to write if we had a big redundant view of all data, try to
define such a view (once you've come up with that view, the
following queries will be very simple one-liners).

Using joins we can also sometimes simplify unwieldly
solutions with subqueries. We saw above that we could find
all nobel laureates who had the longest name in each
category (possibly tied), using the correlated subquery:

In [ ]:
%%sql
SELECT  category, year, name
FROM    nobel AS outer_nobel
WHERE   length(name) = (
            SELECT max(length(name))
            FROM   nobel
            WHERE  category = outer_nobel.category)
ORDER BY category

A nicer way of solving this is to first save the length of
the longest name for each subquery in a temporary table, and
then join this table using the category:

In [ ]:
%%sql
WITH max_category_name_length(category, max_name_length) AS (
  SELECT   category, max(length(name))
  FROM     nobel
  GROUP BY category
)
SELECT year, category, name
FROM   nobel
JOIN   max_category_name_length
USING  (category)
WHERE  length(name) = max_name_length
ORDER BY  category, year, name

In the joined table (`nobel JOIN max_category_name_length`),
each row will get the additional `max_name_length` column,
and it's the max length for the category of the award the
row describes (since we joined using `category`).



## Outer joins

An inner join combines rows in different tables _when there
is a match in the other table_, rows with no corresponding
row in the other table will not turn up in the joined table.

With an _outer join_ we can make sure that every row in one
or both of the tables turn up in the joined table -- in case
there is no match, it will be paired with a row containing
unly `NULL` values (`NULL` is written as `None` in our
notebooks).

As as example, we've seen how to join students and their
applications:

<!---
SELECT s_name, s_id, c_name, major
FROM   students
JOIN   applications
USING  (s_id)
-->

In [ ]:
%%sql


Now, assume we also want to see those students who haven't
applied, we can do that using a _left outer join_:

<!---
SELECT          s_name, s_id, c_name, major
FROM            students
LEFT OUTER JOIN applications
USING           (s_id)
-->

In [ ]:
%%sql


We still get the rows we got before, but now we also get
rows with `NULL`'s in the attributes from the right
relation, if there is no row in the right table which
corresponds to a row in the left table (i.e., the `s_id`
found in the `students` table has no match in the
`applications` table -- the tuples on the left are sometimes
called _dangling tuples_).

By the way, we can omit the `OUTER` keyword:

<!---
SELECT    s_name, s_id, c_name, major
FROM      students
LEFT JOIN applications
USING     (s_id)
-->

In [ ]:
%%sql


There is a 'natural' version of outer joins ...

<!---
SELECT            s_name, s_id, c_name, major
FROM              students
NATURAL LEFT JOIN applications
-->

In [ ]:
%%sql


... but for the same reasons as for inner joins, it's
_much_, _much_ better to explicitly declare which attribute
we're joining (using `USING`).

There is a way to get the same result without inner or outer
joins, but it requires more code:

In [ ]:
%%sql
SELECT      s_name, s.s_id, c_name, major
FROM        students s
CROSS JOIN  applications a
WHERE       s.s_id = a.s_id
UNION
SELECT      s_name, s_id, NULL, NULL
FROM        students
WHERE       s_id NOT IN (SELECT s_id
                         FROM   applications)

Seing this might help you understand what left outer join
actually returns.

Now, the opposite problem, we want to see applications with
no matching students -- of course we could just swap
`students` and `applications` in the query above, but we
could also use a _right outer join_:

In [ ]:
%%sql
SELECT     s_name, s_id, c_name, major
FROM       students
RIGHT JOIN applications USING (s_id)

Right outer joins are not supported in SQLite (yet), though.

There is also a _full outer join_, which combines the left-
and the right outer joins:

In [ ]:
%%sql
SELECT    s_name, s_id, c_name, major
FROM      students
FULL JOIN applications USING (s_id)

Of the outer joins, SQLite only supports left outer, the
other ones are left out :-).

**Exercise:** _Find all 'spurious' applications, i.e.,
applications where the student id doesn't match any
student's id._ Try to solve this exercise in two ways (one
using an outer join, the other using subqueries).

In [ ]:
%%sql


In [ ]:
%%sql


### Problems/Exercises


**Problem:** _How many olympic games have each continent
hosted?_

In [ ]:
%%sql


**Problem:** _Some years no prize was awarded in some
categories, list all such years/categories (we'll only deal
with physics, chemistry, medicine and literature, since
they're the only ones we have in our database...)._

Hint: To solve this problem we want to keep track of all
possible prizes which could have been awarded, assuming all
categories have been awarded since the outset (and indeed
they have). So we want three things:

+ all years when prizes were awarded (in any category)
+ all categories (in our database...)
+ all combinations of years/categories

We can use a cross join to get all combinations:

In [ ]:
%%sql


Once we have all possible combinations, we can use various
techniques to find the missing ones:

In [ ]:
%%sql


**Problem:** _List the three programs (college/major) which
has the highest gpa for the "last accepted" student._
(Assuming the students are accepted in order of decreasing
gpa).

In [ ]:
%%sql


**Exercise:** _When was the first and the latest olympic games in
each continent?_

In [ ]:
%%sql


**Exercise:** _Generate the unwieldly college application
table we started out with above from the three smaller
tables._

In [ ]:
%%sql


**Exercise:** _Recreate the original `olympics` table from the
improved `olympics` and `continents`._

In [ ]:
%%sql


## Instead of lists

One of the 'rules' of relational databases is that the
values in our tables must be primitive, we're not allowed to
have lists or objects as values. This requirement is called
the ["First normal
form"](https://en.wikipedia.org/wiki/First_normal_form), or
"1NF" -- we'll return to 'normal forms' during lectures 7
and 8.

If we were to write a program to keep track of the phone
numbers of our friends, we could write something like this
in Java:

~~~java
class Friend {

    private String name;
    private List<PhoneNumber> phoneNumbers;
    
    ...
}

List<Friend> friends = ...
~~~


A literal translation of this into a table would be
something like:

~~~
name      phone_numbers
----      -------------
Adam      [123456, 654321]
Bodil     [196811]
~~~


but this is in violation of the 'First normal form', so
instead have a new row for each phone number:

~~~
name      phone
----      -----
Adam      123456
Adam      654321
Bodil     196811
~~~


We can ask for Adam's number with:

~~~sql
SELECT phone
FROM   friends
WHERE  name = 'Adam'
~~~


and we'd get both numbers back:

~~~
phone
-----
123456
654321
~~~


Now, assume we also want to save the birthdays of our
friends. We could add a column for birthdays like this:

~~~
name    birthday    phone
----    --------    -----
Adam    2 dec       123456
Adam    2 dec       654321
Bodil   30 nov      196811
~~~


What's the problem with this?

And what if we also want to save our friends' email
addresses:

~~~
name        birthday    phone       email
----        --------    -----       -----
Adam        2 dec       123456      adam@gmail.com
Adam        2 dec       123456      adam@yahoo.com
Adam        2 dec       654321      adam@gmail.com
Adam        2 dec       654321      adam@yahoo.com
Bodil       30 nov      196811      bodil@itu.dk
Bodil       30 nov      196811      bodil@lu.se
Cecilia     9 apr       511235      cecilia@dn.se
Cecilia     9 apr       641587      cecilia@dn.se
David       12 jun      984531      david@gmail.co
David       12 jun      984531      david@hotmail
Emma        11 aug      123456      emma@lu.se
~~~


It may seem silly to have four rows for Adam, but if we
removed one of them, we would loose vital information. If we
removed the first row, and someone asked who had the phone
number 123456 and the email address adam@gmail.com, the
query:

~~~sql
SELECT  name, birthday
FROM    friends
WHERE   phone = '123456' AND email = 'adam@gmail.com'
~~~


would give an empty result.

The problem with the table above is that we have a lot of
redundance, i.e., information is repeated in many places.
This is not only a waste of storage space, it also makes it
much, much harder to work with our table -- insertions,
updates and removals affects many rows, and if we're not
careful, we might leave the table in a corrupted state.

**Exercise:** _The data in the table above would be much
easier to maintain if we split it into several tables --
suggest how to do that._